In [243]:
!pip3 install pandas
!pip3 install numpy
!pip3 install ta
!pip3 install pandas_ta

In [256]:
import pandas as pd
import numpy as np
import ta
from ta.volatility import AverageTrueRange
from ta.trend import MACD, AroonIndicator
from ta.momentum import RSIIndicator
dat=pd.read_csv("/kaggle/input/zelta-hackathon/data_2020_2023/BTC_2019_2023_1month.csv")
data1=dat

def hawkes_process(data: pd.Series, kappa: float):
    assert(kappa > 0.0)
    alpha = np.exp(-kappa)
    arr = data.to_numpy()
    output = np.zeros(len(data))
    output[:] = np.nan
    for i in range(1, len(data)):
        if np.isnan(output[i - 1]):
            output[i] = arr[i]
        else:
            output[i] = output[i - 1] * alpha + arr[i]
    return pd.Series(output, index=data.index) * kappa

def calculate_heikin_ashi(df):
    ha_df = df.copy()
    ha_df['HA_Close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    ha_df['HA_Open'] = (df['open'].shift(1) + df['close'].shift(1)) / 2
    ha_df['HA_High'] = ha_df[['HA_Open', 'HA_Close', 'high']].max(axis=1)
    ha_df['HA_Low'] = ha_df[['HA_Open', 'HA_Close', 'low']].min(axis=1)
    return ha_df[['HA_Open', 'HA_High', 'HA_Low', 'HA_Close']]

def calculate_fibonacci(df, period='swing'):
    high = df['high'].max() if period == 'swing' else df['high'].iloc[-20:].max()
    low = df['low'].min() if period == 'swing' else df['low'].iloc[-20:].min()
    levels = {
        "100%": high,
        "61.8%": high - (high - low) * 0.618,
        "50%": high - (high - low) * 0.5,
        "38.2%": high - (high - low) * 0.382,
        "0%": low
    }
    return levels

def calculate_indicators(df):
    df['RSI'] = RSIIndicator(df['close'], window=14).rsi()
    df['FastVolumeSMA'] = df['volume'].rolling(window=10).mean()
    df['SlowVolumeSMA'] = df['volume'].rolling(window=50).mean()
    df['FastEMA'] = df['close'].ewm(span=9, adjust=False).mean()
    df['SlowEMA'] = df['close'].ewm(span=26, adjust=False).mean()
    df['FastestEMA'] = df['close'].ewm(span=5, adjust=False).mean()
    aroon = ta.trend.AroonIndicator(data['high'], data['low'], window=25)
    df['AroonUp'] = aroon.aroon_up()
    df['AroonDown'] = aroon.aroon_down()
    return df

def detect_weighted_long_signals(df,threshold,kappa):
    ha_df = calculate_heikin_ashi(df)
    df = pd.concat([df, ha_df], axis=1)
    fib_levels = calculate_fibonacci(df)
    df['Near_61.8%'] = np.abs(df['close'] - fib_levels["61.8%"]) <= (fib_levels["61.8%"] * 0.01)
    macd = MACD(df['close'])
    df['MACD'] = macd.macd()
    df['Signal'] = macd.macd_signal()
    df['Bullish_MACD'] = (df['MACD'] > df['Signal']) & (df['MACD'].shift(1) <= df['Signal'].shift(1))
    atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['ATR'] = atr.average_true_range()
    df['ATR_Check'] = df['ATR'] > (df['close'] * 0.015)
    df['norm_volatility'] = (df['ATR'] - df['ATR'].mean()) / df['ATR'].std()
    df['Hawkes_Volatility'] = hawkes_process(df['norm_volatility'], kappa)
    df['Avg_Volume'] = df['volume'].rolling(window=20).mean()
    df['Volume_Check'] = df['volume'] > (df['Avg_Volume'] * 1.5)
    df = calculate_indicators(df)
    df['Score_Heikin_Ashi'] = np.where(df['HA_Open'] == df['HA_Low'], 0.15, 0)
    df['Score_MACD'] = np.where(df['Bullish_MACD'], 0.4, 0)
    df['Score_Fibonacci'] = np.where(df['Near_61.8%'], 0.25, 0)
    df['Score_ATR'] = np.where(df['ATR_Check'], 0.1, 0)
    df['Score_Volume'] = np.where(df['Volume_Check'], 0.1, 0)
    df['Score_RSI'] = np.where((df['RSI'] > 30) & (df['RSI'] < 70), 0.2, 0)
    df['Score_EMA_Crossover'] = np.where(df['FastEMA'] > df['SlowEMA'], 0.3, 0)
    df['Score_Aroon'] = np.where(df['AroonUp'] > df['AroonDown'], 0.2, 0)
    df['Score_Hawkes_Volatility'] = np.where(df['Hawkes_Volatility'] > df['ATR'], 0.25, 0)
    df['Total_Score'] = (df['Score_Heikin_Ashi'] + df['Score_MACD'] + df['Score_Fibonacci'] +
                         df['Score_ATR'] + df['Score_Volume'] + df['Score_RSI'] +
                         df['Score_EMA_Crossover'] + df['Score_Aroon']+df['Score_Hawkes_Volatility'])
    df['Long_Signal'] = df['Total_Score'] >= threshold
    return df

def detect_weighted_short_signals(df, threshold,kappa):
    ha_df = calculate_heikin_ashi(df)
    df = pd.concat([df, ha_df], axis=1)
    fib_levels = calculate_fibonacci(df)
    df['Near_61.8%'] = np.abs(df['close'] - fib_levels["61.8%"]) <= (fib_levels["61.8%"] * 0.01)
    macd = MACD(df['close'])
    df['MACD'] = macd.macd()
    df['Signal'] = macd.macd_signal()
    df['Bearish_MACD'] = (df['MACD'] < df['Signal']) & (df['MACD'].shift(1) >= df['Signal'].shift(1))
    atr = AverageTrueRange(df['high'], df['low'], df['close'], window=14)
    df['ATR'] = atr.average_true_range()
    df['ATR_Check'] = df['ATR'] > (df['close'] * 0.015)
    df['norm_volatility'] = (df['ATR'] - df['ATR'].mean()) / df['ATR'].std()
    df['Hawkes_Volatility'] = hawkes_process(df['norm_volatility'], kappa)
    df['Avg_Volume'] = df['volume'].rolling(window=20).mean()
    df['Volume_Check'] = df['volume'] > (df['Avg_Volume'] * 1.5)
    df = calculate_indicators(df)
    df['Bearish_EMAs'] = (df['FastEMA'] < df['SlowEMA']) & (df['FastestEMA'] < df['FastEMA'])
    df['VWAP_Crossover'] = df['FastestEMA'] < df['FastEMA']
    df['VWAP_Crossover'] = (df['VWAP_Crossover'].shift(1) == False) & (df['VWAP_Crossover'] == True)
    df['RSI_Smoothed'] = df['RSI'].rolling(window=14).mean()
    df['RSI_Low'] = df['RSI_Smoothed'] < 30
    df['Weakening_Downward_Momentum'] = df['AroonDown'] < df['AroonUp']
    df['Score_EMAs'] = np.where(df['Bearish_EMAs'], 0.2, 0)
    df['Score_VWAP_Crossover'] = np.where(df['VWAP_Crossover'], 0.2, 0)
    df['Score_RSI'] = np.where(df['RSI_Low'], 0.2, 0)
    df['Score_Aroon'] = np.where(df['Weakening_Downward_Momentum'], 0.2, 0)
    df['Score_ATR'] = np.where(df['ATR_Check'], 0.1, 0)
    df['Score_Volume'] = np.where(df['Volume_Check'], 0.1, 0)
    df['Score_Hawkes_Volatility'] = np.where(df['Hawkes_Volatility'] > df['ATR'], 0.2, 0)
    df['Total_Score'] = (df['Score_EMAs'] + df['Score_VWAP_Crossover'] + df['Score_RSI'] +
                         df['Score_Aroon'] + df['Score_ATR'] + df['Score_Volume'] + df['Score_Hawkes_Volatility'])
    df['Short_Signal'] = df['Total_Score'] >= threshold
    return df

d1=detect_weighted_long_signals(data1, threshold=0.8,kappa=0.05)
d2=detect_weighted_short_signals(data1, threshold=0.6,kappa=0.05)
long=list(d1['Long_Signal'].to_numpy())
for i in range(1,len(long)):
    if long[i]==True:
        long[i]=1
    else:
        long[i]=0
short=list(d2['Short_Signal'].to_numpy())
for i in range(len(short)):
    if short[i]==True:
        short[i]=-1
    else:
        short[i]=0
signal=[0]*len(long)
for i in range(len(long)):
    signal[i]=short[i]+long[i]
trade_type=[]
for i in range(len(long)):
    if signal[i]==1:
        trade_type.append('Long open')
    elif signal[i]==0:
        trade_type.append('neutral')
    else:
        trade_type.append('short open')

In [257]:
dat['signals']=signal
dat['trade_type']=trade_type
dat

,Unnamed: 0,datetime,open,high,low,close,volume,signals,trade_type
0,0,2019-09-08 16:00:00,10000.00,10000.00,10000.00,10000.00,0.002,0,neutral
1,1,2019-09-08 18:00:00,10344.77,10357.53,10337.43,10340.12,471.659,0,neutral
2,2,2019-09-08 20:00:00,10340.12,10391.90,10324.77,10391.90,1273.030,0,neutral
3,3,2019-09-08 22:00:00,10392.59,10412.65,10366.57,10391.63,1351.600,0,neutral
4,4,2019-09-09 00:00:00,10391.63,10391.63,10391.63,10391.63,0.000,0,neutral
...,...,...,...,...,...,...,...,...,...
18900,18900,2023-12-31 16:00:00,42489.60,42741.60,42465.40,42651.90,12901.812,0,neutral
18901,18901,2023-12-31 18:00:00,42651.90,42766.70,42570.10,42659.90,8594.272,0,neutral
18902,18902,2023-12-31 20:00:00,42659.90,42724.50,42543.30,42558.90,8524.946,0,neutral
18903,18903,2023-12-31 22:00:00,42559.00,42629.50,42083.10,42314.00,23628.811,0,neutral


In [258]:
dat.to_csv('submit3_k=0.05_t=0.8,0.6.csv',index=False)
